In [1]:
from aocd import get_data
from aocd import submit
import unittest

day = 5
year = 2023

def submit_part_a(answer):
    submit(answer, part="a", day=day, year=year)

def submit_part_b(answer):
    submit(answer, part="b", day=day, year=year)

input = get_data(day=day, year=year)

In [2]:
class MapLine:
    def __init__(self, start_including, end_including, step):
        self.start_including = start_including
        self.end_including = end_including
        self.step = step

    def map_from_source(self, source):
        if self.is_source_in_range(source):
            return source + self.step
        return None

    def is_source_in_range(self, source):
        return self.start_including <= source <= self.end_including

    def map_from_source_range(self, source_range):
        start, end = source_range
        return (start + self.step, end + self.step)

    @staticmethod    
    def parse(line):
        destination, source, range_size = [int(value) for value in line.split(" ")]
        return MapLine(source, source + range_size - 1, destination - source)

In [3]:
class MapLineTest(unittest.TestCase):
    def test_parse_1(self):
        mapLine = MapLine.parse("50 98 2")
        self.assertEqual(mapLine.start_including, 98)
        self.assertEqual(mapLine.end_including, 99)
        self.assertEqual(mapLine.step, -48)
        
    def test_map_from_source_1(self):
        mapLine = MapLine.parse("50 98 2")
        self.assertEqual(mapLine.map_from_source(97), None)
        self.assertEqual(mapLine.map_from_source(98), 50)
        self.assertEqual(mapLine.map_from_source(99), 51)
        self.assertEqual(mapLine.map_from_source(100), None)
        
    def test_map_from_source_range_1(self):
        mapLine = MapLine.parse("50 98 2")
        self.assertEqual(mapLine.map_from_source_range((97, 110)), (49, 62))
        self.assertEqual(mapLine.map_from_source_range((48, 148)), (0, 100))
        self.assertEqual(mapLine.map_from_source_range((99, 99)), (51, 51))
        self.assertEqual(mapLine.map_from_source_range((100, 200)), (52, 152))
        

runner = unittest.TextTestRunner(verbosity=3)
res = runner.run(unittest.TestLoader().loadTestsFromTestCase(MapLineTest)) 
assert len(res.failures) == 0

test_map_from_source_1 (__main__.MapLineTest.test_map_from_source_1) ... ok
test_map_from_source_range_1 (__main__.MapLineTest.test_map_from_source_range_1) ... ok
test_parse_1 (__main__.MapLineTest.test_parse_1) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.010s

OK


In [4]:
class Map:
    def __init__(self, map_lines):
        self.map_lines = map_lines

    def map_from_source(self, source):
        for map_line in self.map_lines:
            if map_line.is_source_in_range(source):
                return map_line.map_from_source(source)
        return source

    # Will produce a list of ranges after spliting and mapping
    def map_from_source_ranges(self, source_ranges):
        # Sort input source_ranges order by start then end
        source_ranges = sorted(source_ranges)
        
        # Sort map_lines of map order by start asc
        self.map_lines.sort(key=lambda map_line: map_line.start_including)

        # Prepare Iterators
        source_range_iter = iter(source_ranges)
        map_line_iter = iter(self.map_lines)
        
        target_ranges = []
        # While there is item in the source_ranges list
        current_source_range = next(source_range_iter, None)
        current_map_line = next(map_line_iter, None)
        while current_source_range != None:
            start_including, end_including = current_source_range

            # current_source_range is completely before the map_line range
            # OR no more map_line range to deal with
            if current_map_line == None or end_including < current_map_line.start_including:
                target_ranges.append((start_including, end_including))
                current_source_range = next(source_range_iter, None)
                continue
                
            # current_source_range is completely after the map_line range
            if current_map_line.end_including < start_including:
                current_map_line = next(map_line_iter, None)
                continue

            # Here we know there is a possible intersection between current_source_range and current_map_line range
            # source_start is before the map_line range (and known from previsous conditions: source_end is after current_map_line start)
            if start_including < current_map_line.start_including:
                target_ranges.append((start_including, current_map_line.start_including - 1))
                # This line bellow allow to reach the next condition
                current_source_range = (current_map_line.start_including, end_including)
                continue
                
            # source_start is inside the map_line range
            if current_map_line.start_including <= start_including <= current_map_line.end_including:
                target_ranges.append(current_map_line.map_from_source_range((start_including, min(end_including, current_map_line.end_including))))
                # Have we done work for this source_range ?
                if end_including <= current_map_line.end_including:
                    current_source_range = next(source_range_iter, None)
                else:
                    current_source_range = (min(end_including, current_map_line.end_including) + 1, end_including)
        
        return target_ranges

    @staticmethod
    def parse(lines):
        map_lines = []
        for l in lines:
            map_lines.append(MapLine.parse(l))
        return Map(map_lines)

In [5]:
class MapTest(unittest.TestCase):
    def test_map_from_source_1(self):
        map = Map.parse(["50 98 2", "52 50 48"])
        self.assertEqual(map.map_from_source(10), 10)
        self.assertEqual(map.map_from_source(98), 50)
        self.assertEqual(map.map_from_source(50), 52)
        self.assertEqual(map.map_from_source(90), 92)
        self.assertEqual(map.map_from_source(100), 100)
        
    def test_map_from_source_ranges_1(self):
        map = Map.parse(["50 98 2", "52 50 48"])
        self.assertEqual(map.map_from_source_ranges([(79, 92), (55, 67)]), [(57, 69), (81, 94)])
        
    def test_map_from_source_ranges_2(self):
        map = Map.parse(["0 15 37", "37 52 2", "39 0 15"])
        self.assertEqual(map.map_from_source_ranges([(57, 69), (81, 94)]), [(57, 69), (81, 94)])
        
    def test_map_from_source_ranges_3(self):
        map = Map.parse(["88 18 7", "18 25 70"])
        self.assertEqual(map.map_from_source_ranges([(57, 69), (81, 94)]), [(50, 62), (74, 87)])
        
    def test_map_from_source_ranges_4(self):
        map = Map.parse(["45 77 23", "81 45 19", "68 64 13"])
        self.assertEqual(map.map_from_source_ranges([(50, 62), (74, 87)]), [(86, 98), (78, 80), (45, 55)])
        
    def test_map_from_source_ranges_5(self):
        map = Map.parse(["0 69 1", "1 0 69"])
        self.assertEqual(map.map_from_source_ranges([(86, 98), (78, 80), (45, 55)]), [(46, 56), (78, 80), (86, 98)])
        
    def test_map_from_source_ranges_6(self):
        map = Map.parse(["60 56 37", "56 93 4"])
        self.assertEqual(map.map_from_source_ranges([(46, 56), (78, 80), (86, 98)]), [(46, 55), (60, 60), (82, 84), (90, 96), (56, 59), (97, 98)])
        

runner = unittest.TextTestRunner(verbosity=3)
res = runner.run(unittest.TestLoader().loadTestsFromTestCase(MapTest)) 
assert len(res.failures) == 0

test_map_from_source_1 (__main__.MapTest.test_map_from_source_1) ... ok
test_map_from_source_ranges_1 (__main__.MapTest.test_map_from_source_ranges_1) ... ok
test_map_from_source_ranges_2 (__main__.MapTest.test_map_from_source_ranges_2) ... ok
test_map_from_source_ranges_3 (__main__.MapTest.test_map_from_source_ranges_3) ... ok
test_map_from_source_ranges_4 (__main__.MapTest.test_map_from_source_ranges_4) ... ok
test_map_from_source_ranges_5 (__main__.MapTest.test_map_from_source_ranges_5) ... ok
test_map_from_source_ranges_6 (__main__.MapTest.test_map_from_source_ranges_6) ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.025s

OK


In [6]:
class Almanac:
    def __init__(self, seeds, seed_ranges, maps):
        self.maps = maps
        self.seeds = seeds
        self.seed_ranges = seed_ranges

    def get_locations(self):
        locations = []
        for seed in self.seeds:
            current_seed_value = seed
            for m in self.maps:
                current_seed_value = m.map_from_source(current_seed_value)
            locations.append(current_seed_value)
        return locations
        
    def get_location_ranges(self):
        ranges = self.seed_ranges
        for m in self.maps:
            ranges = m.map_from_source_ranges(ranges)
        return ranges
        
    @staticmethod
    def parse(lines):
        seeds = [int(seed) for seed in lines[0].replace("seeds: ", "").split(" ")]
        seed_ranges = sorted([(seed_range[0], seed_range[0] + seed_range[1] - 1) for seed_range in zip(seeds[::2], seeds[1::2])])
        
        maps = []
        map_lines = []
        # lines[2:] means we don't want to deal with the 2 first lines of the almanac to build maps
        for l in lines[2:]:
            if l == "": # manage new map
                maps.append(Map.parse(map_lines))
            elif l.endswith("map:"):
                map_lines = []
            elif l[0].isdigit:
                map_lines.append(l)
        # manage last map
        maps.append(Map.parse(map_lines))
        return Almanac(seeds, seed_ranges, maps)

In [7]:
class AlmanacTest(unittest.TestCase):
    lines = [
        "seeds: 79 14 55 13"
        "",
        "seed-to-soil map:",
        "50 98 2",
        "52 50 48",
        "",
        "soil-to-fertilizer map:",
        "0 15 37",
        "37 52 2",
        "39 0 15",
        "",
        "fertilizer-to-water map:",
        "49 53 8",
        "0 11 42",
        "42 0 7",
        "57 7 4",
        "",
        "water-to-light map:",
        "88 18 7",
        "18 25 70",
        "",
        "light-to-temperature map:",
        "45 77 23",
        "81 45 19",
        "68 64 13",
        "",
        "temperature-to-humidity map:",
        "0 69 1",
        "1 0 69",
        "",
        "humidity-to-location map:",
        "60 56 37",
        "56 93 4"
    ]

    def test_build_maps_1(self):
        almanac = Almanac.parse(self.lines)
        self.assertEqual(len(almanac.maps), 7)
        self.assertEqual(almanac.maps[0].map_from_source(79), 81)
        self.assertEqual(almanac.maps[0].map_from_source(14), 14)
        self.assertEqual(almanac.maps[0].map_from_source(55), 57)
        self.assertEqual(almanac.maps[0].map_from_source(13), 13)

    def test_get_location_ranges(self):
        almanac = Almanac.parse(self.lines)
        self.assertEqual(almanac.get_location_ranges(), [(46, 55), (60, 60), (82, 84), (86, 89), (94, 96), (56, 59), (97, 98)])


runner = unittest.TextTestRunner(verbosity=3)
res = runner.run(unittest.TestLoader().loadTestsFromTestCase(AlmanacTest)) 
assert len(res.failures) == 0

test_build_maps_1 (__main__.AlmanacTest.test_build_maps_1) ... ok
test_get_location_ranges (__main__.AlmanacTest.test_get_location_ranges) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.006s

OK


In [8]:
%%time
almanac = Almanac.parse(input.split("\n"))
first_answer = min(almanac.get_locations())
print(first_answer)

551761867
CPU times: user 1.26 ms, sys: 56 µs, total: 1.32 ms
Wall time: 1.34 ms


In [9]:
submit_part_a(first_answer)

aocd will not submit that answer again. At 2023-12-05 03:00:29.616248-05:00 you've previously submitted 551761867 and the server responded with:
That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two]


In [10]:
%%time
almanac = Almanac.parse(input.split("\n"))
second_answer = min(almanac.get_location_ranges())[0]
print(second_answer)

57451709
CPU times: user 2.24 ms, sys: 48 µs, total: 2.29 ms
Wall time: 2.32 ms


In [11]:
submit_part_b(second_answer)

aocd will not submit that answer again. At 2023-12-05 16:38:37.495913-05:00 you've previously submitted 57451709 and the server responded with:
That's the right answer!  You are one gold star closer to restoring snow operations.You have completed Day 5! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
